Vary DNA concentration together include GFP DNA concentration

In [3]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result,plot_result_two_state
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model_HC,Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show
import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [4]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.01
T7_DNA.lc =0.01

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 2
GFP_DNA.lc = 2

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 10
Repressor_DNA.lc = 10


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kr = 2;
HC = 2;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,HC,Kr,KTL,KTX,lamb1,lamb2)

end_time = 30

end_cycle = end_time*4+1


In [10]:

def Repressor_model_HC(molecules,t,alpha,beta,K,HC,Kr,KTL,KTX,lamb1,lamb2):
    T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R = molecules
    TX = 1
    TL = 1
    return np.array(
        [
            0,
            alpha*TX*T7_DNA*T7*R/(R+K)*(Kr/(Repressor**HC+Kr)),
            beta*TL*T7_RNA*(R/(R+K)),
            0,
            alpha*TX*GFP_DNA*T7*R/(R+K),
            beta*TL*GFP_RNA*R/(R+K),
            0,
            alpha*TX*Repressor_DNA*T7*R/(R+K),
            0,
            -alpha*TX*T7_DNA*T7*R/(R+K)-alpha*TX*GFP_DNA*T7*R/(R+K)-alpha*TX*Repressor_DNA*T7*R/(R+K)-beta*TL*T7_RNA*R/(R+K)-beta*TL*GFP_RNA*R/(R+K)-beta*TL*Repressor_RNA*R/(R+K),
        ]
    )

In [11]:
for HC in (np.linspace(0,5, 20, endpoint=True)):
    for Kr in  (np.logspace(-3,1, 20, endpoint=True)):
    #Define parameters
        alpha=0.7;
        beta=0.07;
        K=1.0;

        KTL=10;
        KTX=1;
        lamb1=0;
        lamb2=0;

        parameters_list = (alpha,beta,K,HC,Kr,KTL,KTX,lamb1,lamb2)

        end_time = 30

        end_cycle = end_time*4+1


        stored_data = []
        for conc in np.logspace(-4,1, 100, endpoint=True):
            #print(conc)
            #experiment 
            T7_DNA.ic = conc
            T7_DNA.lc = conc
            Repressor_DNA.ic = conc
            Repressor_DNA.lc = conc
            GFP_DNA.ic = conc
            GFP_DNA.lc = conc
            #initial value
            molecules_0 = [None]*len(molecules_list) #create empty list

            for i in range(0,len(molecules_list)):
                molecules_0[i] = molecules_list[i].ic

            #stage 1 kick start
            result_all=[molecules_0] #initial condition


            #DR = 0.2

            #kick start
            dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
            replenish_list = [T7,R]
            result_all,molecules_0 = run_model(Repressor_model_HC,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


            #self regeneration 
            dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
            replenish_list = [R]
            result_all,molecules_0 = run_model(Repressor_model_HC,(4,30),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


            result_all = np.resize(result_all,(end_cycle,10))
            for i in range(0,len(molecules_list)):
                    (molecules_list[i]).result = result_all.transpose()[i]

            #T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
            #plot_result_two_state(GFP.result)
            #print((GFP).result[60]/GFP.pc_result[60])
            stored_data.append((GFP).result[100])


        conc = np.logspace(-4,1, 100, endpoint=True)

        p = figure(width=800, height=400,x_axis_type = "log")
        p.dot(conc,stored_data,size=20)
        p.xaxis.axis_label = 'DNA concentration'
        p.yaxis.axis_label = 'Concentration'

        #show(p)

        file_name = '../data/'+str(round(HC,3)) + str(round(Kr,3)) + '.png'
        bokeh.io.export_png(p, filename=file_name)


In [76]:
conc = np.logspace(-4,1, 100, endpoint=True)

p = figure(width=800, height=400,x_axis_type = "log")
p.dot(conc,stored_data,size=20)
p.xaxis.axis_label = 'DNA concentration'
p.yaxis.axis_label = 'Concentration'

show(p)
bokeh.io.export_png(p, filename="plot.png")


In [12]:
Repressor.result

array([1.00000000e+01, 1.00000000e+01, 8.00000000e+00, 6.40000000e+00,
       5.12000000e+00, 4.09600000e+00, 3.27680000e+00, 2.62144000e+00,
       2.09715200e+00, 1.67772160e+00, 1.34217728e+00, 1.07374182e+00,
       8.58993459e-01, 6.87194767e-01, 5.49755814e-01, 4.39804651e-01,
       3.51843721e-01, 2.81474977e-01, 2.25179981e-01, 1.80143985e-01,
       1.44115188e-01, 1.15292150e-01, 9.22337204e-02, 7.37869763e-02,
       5.90295810e-02, 4.72236648e-02, 3.77789319e-02, 3.02231455e-02,
       2.41785164e-02, 1.93428131e-02, 1.54742505e-02, 1.23794004e-02,
       9.90352031e-03, 7.92281625e-03, 6.33825300e-03, 5.07060240e-03,
       4.05648192e-03, 3.24518554e-03, 2.59614843e-03, 2.07691874e-03,
       1.66153499e-03, 1.32922800e-03, 1.06338240e-03, 8.50705917e-04,
       6.80564734e-04, 5.44451787e-04, 4.35561430e-04, 3.48449144e-04,
       2.78759315e-04, 2.23007452e-04, 1.78405962e-04, 1.42724769e-04,
       1.14179815e-04, 9.13438523e-05, 7.30750819e-05, 5.84600655e-05,
      

In [2]:
str(2)+str(2)

'22'